In [1]:
# url = 'https://www.booking.com/searchresults.en-us.html?ss={}&ssne={}&ssne_untouched={}&label=gen173nr-1FCAEoggI46AdIM1gEaKcCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuALAzp6cBsACAdICJDFhMjc3YWQ4LTY3ZjktNDNhNy05NTI0LTUxMWY3NWQ2ODFmNdgCBeACAQ&sid=2126271f814d7c3f7af3ef0e5ea6d865&aid=304142&lang=en-us&sb=1&src_elem=sb&src=index&dest_id=2469&dest_type=region&checkin={}&checkout={}&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure'

In [133]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import selenium
from selenium.webdriver.common.by import By
import random

In [3]:
from selenium.webdriver.common.keys import Keys

In [4]:
from selenium.webdriver import Chrome

In [5]:
path = '~/user/local/bin/chromedriver'

In [132]:
browser = Chrome()

In [7]:
url = "https://www.booking.com"

In [8]:
browser.get(url)

In [9]:
state_name=browser.find_element(By.NAME, "ss")

In [10]:
state_name.send_keys('New York')

In [11]:
dates = browser.find_element(By.XPATH, "//div[@class='xp__dates-inner']")

In [12]:
dates.click()

In [13]:
Month_next = browser.find_element(By.XPATH,"//div[@data-bui-ref='calendar-next']")

In [14]:
Month_next.click()

In [15]:
check_in = browser.find_element(By.XPATH,"//td[@data-date='2023-02-08']")

In [16]:
check_out = browser.find_element(By.XPATH,"//td[@data-date='2023-02-14']")

In [17]:
check_in.click()

In [18]:
check_out.click()

In [19]:
submitbutton = browser.find_element(By.XPATH, "//button[@type='submit']")

In [20]:
submitbutton.click()

In [60]:
soup = BeautifulSoup(browser.page_source, 'html.parser')

In [61]:
total_pages = int(browser.find_element(By.CSS_SELECTOR, 'div[data-testid="pagination"]  li:last-child').text)
# print(f"Total pages: {total_pages}")

In [62]:
all_detail_links = []

In [63]:
def get_links():
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    all_properties = soup.findAll("div", {"data-testid" : "property-card"})
    for property in all_properties:
        h3 = property.find('h3')
        link = h3.find('a')['href']
        all_detail_links.append(link)

In [64]:
import time

In [ ]:
for current_page in range(1,total_pages+1):
    get_links()
    next_page_btn = browser.find_element(By.XPATH, '//button[contains(@aria-label, "Next page")]')
    next_page_btn.click()
    time.sleep(6)

In [2]:
import json

# create the scraped_hotels folder if you can create different different folder for different state name it would be great

In [130]:
def save_file(data,c):
    with open(f'scraped_hotels/{c}.json', 'w+') as f:
        json.dump(data, f, indent=4)

In [135]:
all_properties = []
c = 0
for link in all_detail_links:
    browser.get(link)
    c += 1
    try:
        soup = BeautifulSoup(browser.page_source,'html.parser')
        hotel_label = soup.find('h2',class_ = 'pp-header__title').text
        location = soup.find('span',class_ = 'hp_address_subtitle').text
        total_images = soup.find('span', class_="bh-photo-grid-thumb-more").text
        rating_block = soup.find("div", {"data-testid" : "review-score-right-component"})
        rating_score = rating_block.findChild().text
        sub_block = rating_block.findChildren()[1]
        rating_label = sub_block.findChild().text
        total_reviews = sub_block.findChildren()[1].text
        best_facility = soup.find('p',class_ = 'best-review-score-label').text
        best_review_score_badge = soup.find('span',class_='review-score-badge').text
        deatil = soup.find('div',class_ = 'hp_desc_main_content').text
        couple_review = soup.find('p',class_ = 'hp-desc-review-highlight').text
        important_facility = soup.find('div',class_ = 'hp_desc_important_facilities').text
        price = soup.find('span',class_ = 'prco-valign-middle-helper').text
    except:
        print(c,link)
        continue
    data = dict(hotel_label=hotel_label,
                location=location,
                total_images=total_images,
                rating_score=rating_score,
                rating_label=rating_label,
                total_reviews=total_reviews,
                best_facility=best_facility,
                best_review_score_badge=best_review_score_badge,
                deatil=deatil,
                couple_review=couple_review,
                important_facility=important_facility,
                price=price,
               source=str(soup))
    save_file(data, c)
    time.sleep(random.randint(3,5))

136 https://www.booking.com/hotel/us/hilton-times-square-new-york.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuALTr8mcBsACAdICJDQzOTBiZWIxLTcxNDQtNDcwNi05NmU4LTVjNGJkYzE2ZmU1N9gCBeACAQ&sid=39d69ef60db6be4413c20e151e506e68&aid=304142&ucfs=1&arphpl=1&checkin=2023-02-08&checkout=2023-02-14&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=10&hapos=135&sr_order=popularity&srpvid=d9d2976b63a500de&srepoch=1670539601&all_sr_blocks=905762401_361666046_2_2_0&highlighted_blocks=905762401_361666046_2_2_0&matching_block_id=905762401_361666046_2_2_0&sr_pri_blocks=905762401_361666046_2_2_0__145883&tpi_r=2&from_sustainable_property_sr=1&from=searchresults&map_fdco=1#hotelTmpl
175 https://www.booking.com/hotel/us/grayson-new-york.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuALTr8mcBsACAdICJDQzOTBiZWIxLTcxNDQtNDcwNi05NmU4LTVjNGJkYzE2ZmU1N9gCBeACAQ&sid=39d69ef60db6be4413c20e151e506e68&aid=304142&ucfs=1&arphpl=1

357 https://www.booking.com/hotel/us/luxe-studio-apt-with-charming-juliet-balcony.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuALTr8mcBsACAdICJDQzOTBiZWIxLTcxNDQtNDcwNi05NmU4LTVjNGJkYzE2ZmU1N9gCBeACAQ&sid=39d69ef60db6be4413c20e151e506e68&aid=304142&ucfs=1&arphpl=1&checkin=2023-02-08&checkout=2023-02-14&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=6&hapos=356&sr_order=popularity&srpvid=d9d2976b63a500de&srepoch=1670539662&all_sr_blocks=921488202_364003938_4_0_0&highlighted_blocks=921488202_364003938_4_0_0&matching_block_id=921488202_364003938_4_0_0&sr_pri_blocks=921488202_364003938_4_0_0__208600&tpi_r=2&from=searchresults&map_fdco=1#hotelTmpl
363 https://www.booking.com/hotel/us/days-inn-broadway.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuALTr8mcBsACAdICJDQzOTBiZWIxLTcxNDQtNDcwNi05NmU4LTVjNGJkYzE2ZmU1N9gCBeACAQ&sid=39d69ef60db6be4413c20e151e506e68&aid=304142&ucfs=1&arphpl=1&checkin=2023-0

404 https://www.booking.com/hotel/us/beautiful-harlem-brownstown-garden-apatment.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuALTr8mcBsACAdICJDQzOTBiZWIxLTcxNDQtNDcwNi05NmU4LTVjNGJkYzE2ZmU1N9gCBeACAQ&sid=39d69ef60db6be4413c20e151e506e68&aid=304142&ucfs=1&arphpl=1&checkin=2023-02-08&checkout=2023-02-14&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=3&hapos=403&sr_order=popularity&srpvid=d9d2976b63a500de&srepoch=1670539676&all_sr_blocks=909691901_363498076_4_0_0&highlighted_blocks=909691901_363498076_4_0_0&matching_block_id=909691901_363498076_4_0_0&sr_pri_blocks=909691901_363498076_4_0_0__132000&from=searchresults&map_fdco=1#hotelTmpl
406 https://www.booking.com/hotel/us/cozy-entire-apartment-close-to-central-park.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuALTr8mcBsACAdICJDQzOTBiZWIxLTcxNDQtNDcwNi05NmU4LTVjNGJkYzE2ZmU1N9gCBeACAQ&sid=39d69ef60db6be4413c20e151e506e68&aid=304142&ucfs=1&arphpl

426 https://www.booking.com/hotel/us/exclusive-3br-blocks-from-times-square.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuALTr8mcBsACAdICJDQzOTBiZWIxLTcxNDQtNDcwNi05NmU4LTVjNGJkYzE2ZmU1N9gCBeACAQ&sid=39d69ef60db6be4413c20e151e506e68&aid=304142&ucfs=1&arphpl=1&checkin=2023-02-08&checkout=2023-02-14&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=25&hapos=425&sr_order=popularity&srpvid=d9d2976b63a500de&srepoch=1670539676&all_sr_blocks=886336902_358200961_0_0_0&highlighted_blocks=886336902_358200961_0_0_0&matching_block_id=886336902_358200961_0_0_0&sr_pri_blocks=886336902_358200961_0_0_0__331020&from=searchresults&map_fdco=1#hotelTmpl
427 https://www.booking.com/hotel/us/townhouse.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuALTr8mcBsACAdICJDQzOTBiZWIxLTcxNDQtNDcwNi05NmU4LTVjNGJkYzE2ZmU1N9gCBeACAQ&sid=39d69ef60db6be4413c20e151e506e68&aid=304142&ucfs=1&arphpl=1&checkin=2023-02-08&checkout=2023-02

444 https://www.booking.com/hotel/us/maison-bella-modern-studio-w-charming-juliet-balcony.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuALTr8mcBsACAdICJDQzOTBiZWIxLTcxNDQtNDcwNi05NmU4LTVjNGJkYzE2ZmU1N9gCBeACAQ&sid=39d69ef60db6be4413c20e151e506e68&aid=304142&ucfs=1&arphpl=1&checkin=2023-02-08&checkout=2023-02-14&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=18&hapos=443&sr_order=popularity&srpvid=d9d2976b63a500de&srepoch=1670539683&all_sr_blocks=881335702_363997320_2_0_0&highlighted_blocks=881335702_363997320_2_0_0&matching_block_id=881335702_363997320_2_0_0&sr_pri_blocks=881335702_363997320_2_0_0__183700&tpi_r=2&from=searchresults&map_fdco=1#hotelTmpl
445 https://www.booking.com/hotel/us/wall-street-journal-new-york.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuALTr8mcBsACAdICJDQzOTBiZWIxLTcxNDQtNDcwNi05NmU4LTVjNGJkYzE2ZmU1N9gCBeACAQ&sid=39d69ef60db6be4413c20e151e506e68&aid=304142&ucfs=1&arp

...

470 https://www.booking.com/hotel/us/steps-to-times-square-magnificent-2br-apartment.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuALTr8mcBsACAdICJDQzOTBiZWIxLTcxNDQtNDcwNi05NmU4LTVjNGJkYzE2ZmU1N9gCBeACAQ&sid=39d69ef60db6be4413c20e151e506e68&aid=304142&ucfs=1&arphpl=1&checkin=2023-02-08&checkout=2023-02-14&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=19&hapos=469&sr_order=popularity&srpvid=d9d2976b63a500de&srepoch=1670539690&all_sr_blocks=886321202_361567740_0_0_0&highlighted_blocks=886321202_361567740_0_0_0&matching_block_id=886321202_361567740_0_0_0&sr_pri_blocks=886321202_361567740_0_0_0__576180&from=searchresults&map_fdco=1#hotelTmpl
471 https://www.booking.com/hotel/us/austin-cove-near-subway-all-shops-restaurants.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuALTr8mcBsACAdICJDQzOTBiZWIxLTcxNDQtNDcwNi05NmU4LTVjNGJkYzE2ZmU1N9gCBeACAQ&sid=39d69ef60db6be4413c20e151e506e68&aid=304142&ucfs=1

482 https://www.booking.com/hotel/us/immaculate-3-bedroom-apartment-at-midtown-east.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuALTr8mcBsACAdICJDQzOTBiZWIxLTcxNDQtNDcwNi05NmU4LTVjNGJkYzE2ZmU1N9gCBeACAQ&sid=39d69ef60db6be4413c20e151e506e68&aid=304142&ucfs=1&arphpl=1&checkin=2023-02-08&checkout=2023-02-14&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=6&hapos=481&sr_order=popularity&srpvid=d9d2976b63a500de&srepoch=1670539697&all_sr_blocks=934393301_365539178_6_0_0&highlighted_blocks=934393301_365539178_6_0_0&matching_block_id=934393301_365539178_6_0_0&sr_pri_blocks=934393301_365539178_6_0_0__519000&from=searchresults&map_fdco=1#hotelTmpl
483 https://www.booking.com/hotel/us/stunning-3bd-apartment-at-midtown-east.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuALTr8mcBsACAdICJDQzOTBiZWIxLTcxNDQtNDcwNi05NmU4LTVjNGJkYzE2ZmU1N9gCBeACAQ&sid=39d69ef60db6be4413c20e151e506e68&aid=304142&ucfs=1&arphpl=1

494 https://www.booking.com/hotel/us/great-studio-apts.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuALTr8mcBsACAdICJDQzOTBiZWIxLTcxNDQtNDcwNi05NmU4LTVjNGJkYzE2ZmU1N9gCBeACAQ&sid=39d69ef60db6be4413c20e151e506e68&aid=304142&ucfs=1&arphpl=1&checkin=2023-02-08&checkout=2023-02-14&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=18&hapos=493&sr_order=popularity&srpvid=d9d2976b63a500de&srepoch=1670539697&all_sr_blocks=571927501_363621225_2_0_0&highlighted_blocks=571927501_363621225_2_0_0&matching_block_id=571927501_363621225_2_0_0&sr_pri_blocks=571927501_363621225_2_0_0__299400&from=searchresults&map_fdco=1#hotelTmpl
495 https://www.booking.com/hotel/us/modish-2br-apartment-in-nyc.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuALTr8mcBsACAdICJDQzOTBiZWIxLTcxNDQtNDcwNi05NmU4LTVjNGJkYzE2ZmU1N9gCBeACAQ&sid=39d69ef60db6be4413c20e151e506e68&aid=304142&ucfs=1&arphpl=1&checkin=2023-02-08&checkout=2023-02-14

506 https://www.booking.com/hotel/us/pink-paradise-pavilion-nyc-most-insta-worthy-spot.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuALTr8mcBsACAdICJDQzOTBiZWIxLTcxNDQtNDcwNi05NmU4LTVjNGJkYzE2ZmU1N9gCBeACAQ&sid=39d69ef60db6be4413c20e151e506e68&aid=304142&ucfs=1&arphpl=1&checkin=2023-02-08&checkout=2023-02-14&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=5&hapos=505&sr_order=popularity&srpvid=d9d2976b63a500de&srepoch=1670539703&all_sr_blocks=902538801_361579743_2_0_0&highlighted_blocks=902538801_361579743_2_0_0&matching_block_id=902538801_361579743_2_0_0&sr_pri_blocks=902538801_361579743_2_0_0__427680&from=searchresults&map_fdco=1#hotelTmpl
507 https://www.booking.com/hotel/us/apartment-new-york-new-york3.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuALTr8mcBsACAdICJDQzOTBiZWIxLTcxNDQtNDcwNi05NmU4LTVjNGJkYzE2ZmU1N9gCBeACAQ&sid=39d69ef60db6be4413c20e151e506e68&aid=304142&ucfs=1&arphpl=1&checki

518 https://www.booking.com/hotel/us/4bedroom-apartment-dg-style-on-manhattan.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuALTr8mcBsACAdICJDQzOTBiZWIxLTcxNDQtNDcwNi05NmU4LTVjNGJkYzE2ZmU1N9gCBeACAQ&sid=39d69ef60db6be4413c20e151e506e68&aid=304142&ucfs=1&arphpl=1&checkin=2023-02-08&checkout=2023-02-14&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=17&hapos=517&sr_order=popularity&srpvid=d9d2976b63a500de&srepoch=1670539703&all_sr_blocks=918590101_363436170_2_0_0&highlighted_blocks=918590101_363436170_2_0_0&matching_block_id=918590101_363436170_2_0_0&sr_pri_blocks=918590101_363436170_2_0_0__437400&from=searchresults&map_fdco=1#hotelTmpl
519 https://www.booking.com/hotel/us/2-bedroom-in-upper-west-side.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuALTr8mcBsACAdICJDQzOTBiZWIxLTcxNDQtNDcwNi05NmU4LTVjNGJkYzE2ZmU1N9gCBeACAQ&sid=39d69ef60db6be4413c20e151e506e68&aid=304142&ucfs=1&arphpl=1&checkin=2023-0

In [3]:
import os

In [4]:
files = os.listdir('scraped_hotels/')

In [5]:
def load_file(filename):
    with open(filename) as f:
        data = json.loads(f.read())
    return data

In [6]:
hotels = [load_file('scraped_hotels/' + file) for file in files]

In [8]:
df = pd.DataFrame(hotels)

In [10]:
df.drop('source', axis=1, inplace=True)

In [11]:
df

,hotel_label,location,total_images,rating_score,rating_label,total_reviews,best_facility,best_review_score_badge,deatil,couple_review,important_facility,price
0,Redford Hotel,"\n136 Ludlow Street, Lower East Side, New York...",\n\n\n+27 photos\n\n\n,6.9,Pleasant,351 reviews,\nGreat location!\n,\n8.5\n,\n\n\n\n\n\nYou're eligible for a Genius disco...,\nCouples in particular like the location – th...,\n\nMost popular amenities\n\n\n\nNon-smoking ...,\n$841\n
1,Aloft Manhattan Downtown - Financial District,"\n49-53 Ann Street, Wall Street - Financial Di...",\n\n\n+29 photos\n\n\n,7.5,Good,"1,070 reviews",\nGreat location!\n,\n8.5\n,"\n\n\n\n\n\nLocated in the Financial District,...",\nCouples in particular like the location – th...,\n\nMost popular amenities\n\n\n\nNon-smoking ...,\n$819\n
2,The Peninsula New York,"\n700 Fifth Avenue at 55th Street, New York, N...",\n\n\n+57 photos\n\n\n,8.8,Excellent,599 reviews,\nExcellent location!\n,\n9.7\n,\n\n\n\nStay in the heart of New York\n\n–\n\n...,\nCouples in particular like the location – th...,\n\nMost popular amenities\n\n\n\n1 swimming p...,"\n$4,056\n"
3,Fully Furnished Entire Floor Apartment in Hist...,"\n517 Manhattan Avenue, apartment #1, Harlem, ...",\n\n\n+17 photos\n\n\n,9.1,Wonderful,43 reviews,\nFree WiFi\n,\n10\n,\n\n\n\n\n\nYou're eligible for a Genius disco...,\nCouples in particular like the location – th...,\n\nMost popular amenities\n\n\n\nFree WiFi\n\...,\n$900\n
4,"The Bernic Hotel New York City,Tapestry Collec...","\n145 East 47th Street, Midtown East, New York...",\n\n\n+49 photos\n\n\n,8.2,Very Good,"1,205 reviews",\nStaff\n,\n8.8\n,\n\n\n\nStay in the heart of New York\n\n–\n\n...,\nCouples in particular like the location – th...,\n\nMost popular amenities\n\n\n\nNon-smoking ...,"\n$1,044\n"
...,...,...,...,...,...,...,...,...,...,...,...,...
395,YOTEL New York Times Square,"\n570 10th Avenue, Hell's Kitchen, New York, N...",\n\n\n+45 photos\n\n\n,7.2,Good,"3,702 reviews",\nGreat location!\n,\n8.6\n,\n\n\n\nStay in the heart of New York\n\n–\n\n...,\nCouples in particular like the location – th...,\n\nMost popular amenities\n\n\n\nNon-smoking ...,\n$843\n
396,The Westin New York Grand Central,"\n212 East 42nd Street, Murray Hill, New York,...",\n\n\n+48 photos\n\n\n,8.0,Very Good,507 reviews,\nExcellent location!\n,\n9.2\n,\n\n\n\nStay in the heart of New York\n\n–\n\n...,\nCouples in particular like the location – th...,\n\nMost popular amenities\n\n\n\nNon-smoking ...,"\n$1,284\n"
397,Moxy NYC Downtown,"\n26 Ann Street, Wall Street - Financial Distr...",\n\n\n+53 photos\n\n\n,8.4,Very Good,598 reviews,\nExcellent location!\n,\n9.0\n,\n\n\n\n\n\nLocated in the heart of New York’s...,\nCouples in particular like the location – th...,\n\nMost popular amenities\n\n\n\nNon-smoking ...,"\n$1,004\n"
398,Executive Hotel Le Soleil New York,"\n38 West 36th Street, New York, NY 10018, Uni...",\n\n\n+47 photos\n\n\n,8.2,Very Good,"2,368 reviews",\nExcellent location!\n,\n9.3\n,\n\n\n\n\n\nYou're eligible for a Genius disco...,\nCouples in particular like the location – th...,\n\nMost popular amenities\n\n\n\nNon-smoking ...,"\n$1,414\n"


# need to put the clean code here like remove the \n from both side and all then run last one for getting the data into csv

In [ ]:
df.to_csv('scraped_hotels.csv')